## Putting genetic features into a model readable format



**This code takes the genetic features in model_elements_and_coverages.json (produced in the rules based resistance prediction and the converts them into a model readable output which has**

1. For each beta-lactamase, which bush jacoby class it is in and what its copy number is (e.g. blaTEM_buj2b=1.23 implies the sample contains a class 2b blaTEM geneat DNA copy number 1.23
2. Whether the sample contains mutated ampC promoters of blaTEM promoters (1/0 , 1 if present, 0 otherwise)
3. Whether the sample contains "non-functioning" ompC/F genes, (1 if present, 0 otherwise)

**This is then converted to a useful long format which incorporates the MIC data from **
1. bd_phoenix_mics.csv
2. agar_dilution_raw_mics.json

**Lastly it also adds MLSTs to the data , which are categorised for the 3 most common mlsts in the agar dilution dataset for testing in the model.**

#### To run this code you need.
- Numpy
- Pandas


In [1]:
# Required modules
import pandas as pd
import numpy as np
import csv
import json


#### So the first step is to define beta-lactamase categories

Required files
1. exclusions.csv => List of 11 exclusions from prediction
2. mlsts_plus_ids.csv =>  MLSTS of all samples
3. model_elements_and_coverages.json => Model elements and coverage used as the raw genetic features



In [ ]:
# Reading in data

with open("model_elements_and_coverages.json", "r") as f:
    model_start = json.load(f)
genes = [j.split(".")[0] for j in list(set([k for g in model_start.keys() for k in model_start[g]['blm'].keys()]))]
blms = [i.replace("TEM-1D", 'TEM-1') for i in genes if "bla" in i]
guuids = list(model_start.keys())
mlsts = pd.read_csv("mlsts.csv", index_col=0)
agar_guuids = [k for k in mlsts.index if mlsts.loc[k].complete_agardil == 1]
exclusions = pd.read_csv("exclusions.csv", index_col=0)


# Defining classification categories.

#Families
TEM = ['blaTEM-1','blaTEM-135', 'blaTEM-206', 'blaTEM-149', 
       'blaTEM-150', 'blaTEM-156', 'blaTEM-190', 'blaTEM-198',
       'blaTEM-207', 'blaTEM-208', 'blaTEM-209', 
       'blaTEM-30', 'blaTEM-33', 'blaTEM-34', 'blaTEM-71' ]
CTXM = ['blaCTX-M-1', 'blaCTX-M-132', 'blaCTX-M-14b', 
        'blaCTX-M-15', 'blaCTX-M-27', 'blaCTX-M-3',
        'blaCTX-M-45', 'blaCTX-M-55', 'blaCTX-M-9']
SHV = ['blaSHV-12','blaSHV-11']
OXA = ['blaOXA-1','blaOXA-48' ]
CMY = ['blaCMY-99','blaCMY-2','blaCMY-60' ]
ACT = ['blaACT-16']
LAT = ['blaLAT-1']

#BUSH Jacoby

buj1 = ['blaCMY-2', 'blaACT-16','blaCMY-99', 'blaCMY-60', 'blaLAT-1']
buj2b= ['blaTEM-1','blaSHV-11', 'blaTEM-135']
buj2be = ['blaCTX-M-15','blaCTX-M-14b', 'blaCTX-M-27',
          'blaTEM-209','blaTEM-207' , 'blaTEM-190', 'blaCTX-M-1',
          'blaSHV-12', 'blaCTX-M-55', 'blaCTX-M-45','blaCTX-M-9' ,
          'blaTEM-156', 'blaTEM-149','blaTEM-198','blaCTX-M-132' ,
         'blaCTX-M-3','blaTEM-150','blaTEM-208'  ]
buj2br = ['blaTEM-34','blaTEM-30', 'blaTEM-33', 'blaTEM-71', 'blaTEM-206']
buj2d = ['blaOXA-1']
buj2df = ['blaOXA-48']

# Checking these classes
bush_jacoby_classes = {"buj1":buj1, 'buj2b':buj2b, 'buj2be':buj2be, 
                       'buj2br':buj2br, 'buj2d':buj2d, 'buj2df':buj2df}
bush_jacoby_all = buj1 + buj2b + buj2be + buj2br + buj2d + buj2df
families = {"TEM":TEM, "CTXM":CTXM, "SHV":SHV, "OXA":OXA, "CMY":CMY, 
            "ACT":ACT, "LAT":LAT}
families_all = TEM + CTXM + SHV + OXA + CMY + ACT + LAT
for i in blms:
    assert i in families_all
#Creating intersection lists
combined = {}
for i in list(families.keys()):
    for j in list(bush_jacoby_classes.keys()):
        x = sorted(list(set(bush_jacoby_classes[j])& set(families[i])))
        if x != []:
            combined[i+'_'+j] = x
    combined[i + "_OTHER"] = []
    
# Cats with > 10 isolates
individual_cats = ['TEM_buj2b', 'CTXM_buj2be', 'SHV_buj2b', 'OXA_buj2d']
other_cats = [i for i in combined if i not in individual_cats]



#### Now for each isolate we then approtion categories, 

This is done using the model_cat class. This basically reads everything in , then runs make_cat to actually perform the categorization. the make_output function then actually makes the output required for an intermediate file which contains model categories alone (not merged with MICs)


**NOTE:**
There are 2 important steps which deal with atypical cases to highlight
1. One blaSHV in AMC_926 had its (outlier) copy number truncated to reduce its effect in the overall models
2. 11 samples in the "testing" dataset have had some of their model components set to 0 as they were not seen in the training set, and hence these were assigned an effect of 0

Both of these changes happen in the make_cat function



In [9]:



# Just a useful function
def convert_bool(s):
    if s == False:
        return 0
    else:
        return 1
    
# Now that we have set up a way to categorize things
# Next step is to then code in actually categorizing each sample.

class model_cat:
    
    def __init__(self, guuid):
        self.guuid = guuid
        self.mlst = mlsts.loc[self.guuid].mlst
        self.mlst_cat = mlsts.loc[self.guuid].mlst_cat
        self.mlst_131 = mlsts.loc[self.guuid].mlst_131
        self.mlst_95 = mlsts.loc[self.guuid].mlst_95
        self.mlst_73 = mlsts.loc[self.guuid].mlst_73
        self.mlst_69 = mlsts.loc[self.guuid].mlst_69
        self.complete_agardil = mlsts.loc[self.guuid].complete_agardil
        self.exclusion = convert_bool(self.guuid in exclusions.index)
        self.dict = model_start[self.guuid]
        self.dict_rename = {k.split(".")[0].replace("TEM-1D", 'TEM-1'):self.dict['blm'][k] for k in self.dict['blm'].keys()}
        self.blms = [x.split(".")[0].replace("TEM-1D", 'TEM-1') for x in self.dict['blm'].keys() if x.split(".")[0].replace("TEM-1D", 'TEM-1') in blms]
        self.agar_dil = (self.guuid in agar_guuids)
        self.model_cats = self.make_cat()
        
    def make_cat(self):
        # This is what actually makes components, and basically it does this by deciding what blm class the component is
        # Note the actual categories represent the commonest elements seen in our dataset, as other categories each 
        # (e.g. IRT tem in the agar dilution dataset) are too rare to do anything other than put into an other category.
        # For category components in agar dil data see final table
        cats = {k:0 for k in combined.keys()}
        cats_final = {k:0 for k in ['TEM_buj2b', 'CTXM_buj2be', 'SHV_buj2b', 'OXA_buj2d', 'OTHER_bla']}        
        for k in combined.keys():
            for b in self.blms:
                if b in combined[k]:
                    cats[k] += self.dict_rename[b]
        for k in cats.keys():
            if k in cats_final.keys():
                cats_final[k] = cats[k]
            else:
                if cats[k] != 0:
                    cats_final['OTHER_bla'] = 1
        # So this categorises everything, however, in the "test set" there are several non-seen elements
        # whose effects cannot be reliably estimated using the "training set data".
        # These have been categorised here, however, given their effect are more conservatively described as 0 as
        # They are unknown, they are set to zero.
        # This is done on a per sample basis. (see exclusions doc for more detail)
        if self.exclusion == 1:
            cats_final[exclusions.loc[self.guuid].element] = 0
        # In addition we truncate one outlier blaSHV copy number (see supplementary methods)
        if self.guuid == "AMC_926":
            cats_final['SHV_buj2b'] = 16.267611
        return cats_final

    def make_output(self):
        # This then writes a sample specific line in the model categorization
        output = {}
        output['guuid'] = self.guuid
        for i in self.model_cats.keys():
            output["bla" + i.lower()] = self.model_cats[i]
        output['ampcprother_sigampcpr'] = convert_bool(self.dict['ampc'])
        output['tempromter_sigtem'] = convert_bool(self.dict['temp'])
        output['ompnon_functioning'] = convert_bool(self.dict['nfomp'])
        output['inagar'] = self.agar_dil
        output["exclusion"] = self.exclusion
        output['mlst'] = self.mlst
        output['mlst_cat'] = self.mlst_cat
        output["mlst_131"] = self.mlst_131
        output["mlst_95"] = self.mlst_95
        output["mlst_73"] = self.mlst_73
        output["mlst_69"] = self.mlst_69
        output["complete_agardil"] = self.complete_agardil
        return output
        
# Actually carrying out the categorization
categories = {k:0 for k in combined.keys()}

with open("model_categories.csv", "w") as f:
    writer = csv.writer(f, delimiter = ",")
    writer.writerow(['guuid', 'blatem_buj2b', 'blactxm_buj2be', 'blashv_buj2b', 'blaoxa_buj2d', 'blaother_bla', 'ampcprother_sigampcpr', 'temprother_sigtem', 'ompnon_functioning', 'inagar', "prediction_exclusion", "mlst", "mlst_cat", "mlst_131", "mlst_95", "mlst_73", "mlst_69", "complete_agardil"])
    for i in guuids:
        x = model_cat(i)
        writer.writerow([x.make_output()[i] for i in 
                         ['guuid', 'blatem_buj2b', 'blactxm_buj2be', 'blashv_buj2b', 'blaoxa_buj2d', 'blaother_bla', 'ampcprother_sigampcpr', 'tempromter_sigtem', 'ompnon_functioning', 'inagar', "exclusion", "mlst", "mlst_cat", "mlst_131", "mlst_95", "mlst_73", "mlst_69", "complete_agardil"]])



#### Next step is to merge these with the agar dilution MIC data


This merges the intermediate file (model_categories.csv) with all the phenotype data, (bd phoenix and agar dilution) It then combines these and writes them out into the final model_base.csv for input to the prediction file

In [15]:
# Next we merge this with the routine and agar dilution MIC data


#Reading in data

cat_df = pd.read_csv("model_categories.csv", index_col = 0)
mlsts = pd.read_csv("mlsts.csv")
guuids = list(cat_df.index)
with open("agar_dilution_raw_mics.json" , "r")  as f:
    mics_df = json.load(f)
routine_lab = pd.read_csv('bd_phoenix_mics.csv', index_col = 0)



# Functions to convert MICs to human readable
def convert_clsi_hrf(no):
    no = no.lstrip("<=")
    no = no.lstrip(">")
    no = int(float(no))
    if no == 2.0:
        start = "<="
    elif no == 256.0:
        start = ">"
    else:
        start = ""
    x = start +  str(no) +"/" +str(int(no/2))
    return x

def convert_eucast_hrf(no):
    no = no.lstrip("<=")
    no = no.lstrip(">")
    no = int(float(no))
    if no == 2.0:
        start = "<="
    elif no == 256.0:
        start = ">"
    else:
        start = ""
    x = start +  str(no) +"/" +str(2)
    return x

with open("model_base.csv", "w") as f:
    writer = csv.writer(f)
    header_line = ['guuid', 'blatem_buj2b', 'blactxm_buj2be', 'blashv_buj2b', 'blaoxa_buj2d', 'blaother_bla',
                   'ampcprother_sigampcpr', 'temprother_sigtem', 'ompnon_functioning', 'inagar', 
                   'complete_agardil', 'mlst', 'mlst_cat', "mlst_131", "mlst_95","mlst_73", "mlst_69", 
                   'coa_mic', "prediction_exclusion"]
    header_line_1 = header_line + ['test_type', 'mic', 'untransformed_mic', 'batch_no']
    writer.writerow(header_line_1)
    for i in guuids:
        i_out = {}
        for k in cat_df.columns.values:
            i_out[k] = cat_df.loc[i][k]
        for k in routine_lab.columns.values:
            i_out[k] = routine_lab.loc[i][k]
        if i not in mics_df.keys(): # IE those NOT tested by agar dilution
            j_list = []
            for k in header_line:
                if k == "guuid":
                    j_list.append(i)
                else:
                    j_list.append(i_out[k])
            # Next we add the test type , (which for all BD phoenix experiments is best approximated as EUCAST)  ie test type = 1
            j_list.append(1) #test type , note we use this in the "prediction step"
            j_list.append("") # mic result -> NONE HERE
            j_list.append("") #untransformed mic
            j_list.append("") #batch no
            writer.writerow(j_list)
        else:
            clsi_results = mics_df[i]['clsi']
            for key in mics_df[i]['clsi'].keys():
                batch_no = int(key.split("_")[-1])
                j_list = []
                for k in header_line:
                    if k == "guuid":
                        j_list.append(i)
                    else:
                        j_list.append(i_out[k])
                j_list.append(0)
                # Note here, the MIC used in the model is actually the one below the limit of detection (i.e. 1.0  <=2/2) and the same aplies at the top end. (>128 becomes 256)
                # See Croghan, C AND P P. Egeghy. METHODS OF DEALING WITH VALUES BELOW THE LIMIT OF DETECTION USING SAS. Presented at Southeastern SAS User Group, St. Petersburg, FL, September 22-24, 2003.
                j_list.append(np.log2(mics_df[i]['clsi'][key]['MIC_FOR_MODEL_ACCOUNTING_FOR_CENSORING']))
                j_list.append(convert_clsi_hrf(mics_df[i]['clsi'][key]['RECORDED_MIC']))
                j_list.append(batch_no)
                writer.writerow(j_list)
            eucast_results = mics_df[i]['eucast']
            for key in mics_df[i]['eucast'].keys():
                batch_no = int(key.split("_")[-1])
                j_list = []
                for k in header_line:
                    if k == "guuid":
                        j_list.append(i)
                    else:
                        j_list.append(i_out[k])
                j_list.append(1)
                # Note here, the MIC used in the model is actually the one below the limit of detection (i.e. 1.0  <=2/2) and the same aplies at the top end. (>128 becomes 256)
                # See Croghan, C AND P P. Egeghy. METHODS OF DEALING WITH VALUES BELOW THE LIMIT OF DETECTION USING SAS. Presented at Southeastern SAS User Group, St. Petersburg, FL, September 22-24, 2003.
                j_list.append(np.log2(mics_df[i]['eucast'][key]['MIC_FOR_MODEL_ACCOUNTING_FOR_CENSORING']))
                j_list.append(convert_eucast_hrf(mics_df[i]['eucast'][key]['RECORDED_MIC']))
                j_list.append(batch_no)
                writer.writerow(j_list)


At the end of this process we have a file, model_base.csv, from which to build our mixed models
**From here on in we switch to the STATA code for model fitting and predictions**

### NOTE IN THE FINAL DATASET, COAMIC IS THE BD PHOENIX MIC, MIC is the AGAR DILUTION MIC. This is will corrected in the STATA do file